# Soil Summary by Parcel

Getting soil characteristic summary stats for parcels in Allegeny County using:

* data from the County's existing open ArcGIS Online resources.
* non-Esri Python tooling (i.e., not the ArcGIS API for Python)

---

*Notebook prep*

In [1]:
import json
from copy import copy
# dependencies
import requests
import geopandas as gpd
import fiona
import petl as etl
# notebook only:
import folium

## Resources

In [2]:
# the parcel for Highland Park (a complex multi-polygon with holes)
alco_parcel_pin = "0082H00001000002" 

# URLs for parcel and soil feature services, respectively
alco_parcel_url = "https://gisdata.alleghenycounty.us/arcgis/rest/services/OPENDATA/Parcels/MapServer/0/query"
alco_soils_url = "https://services1.arcgis.com/vdNDkVykv9vEWFX4/arcgis/rest/services/Soils/FeatureServer/0/query"

# NAD 83 PA State Plane South. Units in feet
epsg_code = 2272 

## Get the parcel as `geojson`

In [3]:
parcel_geojson = requests.get(
    alco_parcel_url,
    params=dict(
        where="PIN='{0}'".format(alco_parcel_pin),
        outFields='PIN',
        returnGeometry='true',
        outSR=epsg_code,
        f='json'
    )
)

In [4]:
if 'error' in parcel_geojson.json().keys():
    print(json.dumps(parcel_geojson.json(), indent=2))

### ...and read into a Pandas GeoDataFrame

In [5]:
with fiona.BytesCollection(bytes(parcel_geojson.content)) as f:
    parcels_gdf = gpd.GeoDataFrame.from_features(f, crs=f.crs)

print(parcels_gdf.head())

                PIN                                           geometry
0  0082H00001000002  (POLYGON ((1366690.120953456 420745.3951480538...


From that, get the bounding box, which will be used to query the soils data:

In [6]:
parcels_gdf.bounds

,minx,miny,maxx,maxy
0,1.362875e+06,420538.49943,1.369073e+06,428067.736652


In [7]:
parcel_bbox = parcels_gdf.iloc[0]['geometry'].bounds
# the bbox as a comma-separated string, which is how we'll need to use it below
','.join([str(x) for x in parcel_bbox])

'1362874.5757152587,420538.49942983687,1369072.6255371273,428067.73665204644'

## Get the soils data as `geojson`, where it intersects the parcel.

By using the envelope of the parcel in the query to the soils feature service, we keep the query response light and fast.

In [8]:
soil_geojson = requests.get(
    alco_soils_url,
    params=dict(
        where="1=1",
        outFields='SOIL_CODE,CLASS,SUBCLASS',
        returnGeometry='true',
        geometry=','.join([str(x) for x in parcel_bbox]),
        inSR=epsg_code,
        geometryType='esriGeometryEnvelope',
        spatialRel='esriSpatialRelIntersects',
        outSR=epsg_code,
        f='geojson'
    )
)


with fiona.BytesCollection(bytes(soil_geojson.content)) as f:
    soils_gdf = gpd.GeoDataFrame.from_features(f, crs=f.crs)

print(soils_gdf.head())


   CLASS SOIL_CODE SUBCLASS                                           geometry
0  Other       UCE           POLYGON ((1366539.49999961 426184.906251109, 1...
1  Other       UCD           POLYGON ((1368125.24999962 422837.625001107, 1...
2      3       GlC   Medium  POLYGON ((1369787.1249685 424526.78112561, 136...
3  Other       UCD           POLYGON ((1368236.37499962 421707.1562511, 136...
4  Other       UCD           POLYGON ((1365684.37499962 427649.812501096, 1...


## Intersect the geometries and dissolve results

In [9]:
soil_fields = ['SOIL_CODE', 'CLASS', 'SUBCLASS', 'geometry']
dissolve_fields = soil_fields[:-1]
intersected = gpd\
    .overlay(parcels_gdf[['geometry']], soils_gdf[soil_fields], how='intersection')\
    .dissolve(by=dissolve_fields, aggfunc='sum')
    #.dissolve(by='SOIL_CODE', aggfunc='sum')

intersected

,,,geometry
SOIL_CODE,CLASS,SUBCLASS,
GQF,Other,,(POLYGON ((1367011.000416636 423464.4990123659...
GSF,Other,,(POLYGON ((1366475.375317037 426671.0656723827...
GlC,3,Medium,"POLYGON ((1369058.750418246 425203.9990729243,..."
GpB,3,High,"POLYGON ((1368776.148806929 426009.0737125015,..."
GpD,Other,,"(POLYGON ((1365844.905197708 426705.214238181,..."
UB,Other,,"(POLYGON ((1367954.87509571 421595.3741577417,..."
UCB,Other,,"(POLYGON ((1368294.13279247 422256.8451476062,..."
UCD,Other,,"(POLYGON ((1366180.664091966 420682.658891024,..."
UCE,Other,,(POLYGON ((1368289.125031918 422200.9658620805...


## Generate a summary stats table

In [10]:
intersected.area

SOIL_CODE  CLASS  SUBCLASS
GQF        Other              6.034193e+06
GSF        Other              3.528767e+05
GlC        3      Medium      5.318081e+04
GpB        3      High        2.225433e+05
GpD        Other              1.373719e+06
UB         Other              1.537416e+06
UCB        Other              1.388790e+06
UCD        Other              1.361529e+06
UCE        Other              4.172916e+05
URB        Other              1.473164e+06
W          Other              1.621893e+06
dtype: float64

In [11]:
# helper function for reading the dissolved dataframe into tabular columns
def ea(ks, v):
    i = list(ks)
    i.append(v)
    return i

In [12]:
rows = []
header = copy(dissolve_fields)
header.append('area')
rows = [ea(k, v) for k, v in intersected.area.to_dict().items()]
rows.insert(0, header)
print(rows)
table = etl.setheader(rows, header)

[['SOIL_CODE', 'CLASS', 'SUBCLASS', 'area'], ['GQF', 'Other', '', 6034192.801922443], ['GSF', 'Other', '', 352876.74361696845], ['GlC', '3', 'Medium', 53180.80732686039], ['GpB', '3', 'High', 222543.28710910177], ['GpD', 'Other', '', 1373719.3657268938], ['UB', 'Other', '', 1537416.1292352788], ['UCB', 'Other', '', 1388790.0310212607], ['UCD', 'Other', '', 1361528.8281820714], ['UCE', 'Other', '', 417291.5928367517], ['URB', 'Other', '', 1473163.9603918842], ['W', 'Other', '', 1621893.1445283026]]


In [13]:
etl.vis.displayall(table)

SOIL_CODE,CLASS,SUBCLASS,area
GQF,Other,,6034192.801922443
GSF,Other,,352876.74361696845
GlC,3,Medium,53180.80732686039
GpB,3,High,222543.28710910177
GpD,Other,,1373719.3657268938
UB,Other,,1537416.1292352788
UCB,Other,,1388790.0310212607
UCD,Other,,1361528.8281820714
UCE,Other,,417291.5928367517
URB,Other,,1473163.9603918842


## Mimic a "response" `JSON` object:

In [14]:
response = {
    "parcel_pin": alco_parcel_pin,
    "soils": list(etl.dicts(table))
}
response

{'parcel_pin': '0082H00001000002',
 'soils': [{'SOIL_CODE': 'GQF',
   'CLASS': 'Other',
   'SUBCLASS': '',
   'area': 6034192.801922443},
  {'SOIL_CODE': 'GSF',
   'CLASS': 'Other',
   'SUBCLASS': '',
   'area': 352876.74361696845},
  {'SOIL_CODE': 'GlC',
   'CLASS': '3',
   'SUBCLASS': 'Medium',
   'area': 53180.80732686039},
  {'SOIL_CODE': 'GpB',
   'CLASS': '3',
   'SUBCLASS': 'High',
   'area': 222543.28710910177},
  {'SOIL_CODE': 'GpD',
   'CLASS': 'Other',
   'SUBCLASS': '',
   'area': 1373719.3657268938},
  {'SOIL_CODE': 'UB',
   'CLASS': 'Other',
   'SUBCLASS': '',
   'area': 1537416.1292352788},
  {'SOIL_CODE': 'UCB',
   'CLASS': 'Other',
   'SUBCLASS': '',
   'area': 1388790.0310212607},
  {'SOIL_CODE': 'UCD',
   'CLASS': 'Other',
   'SUBCLASS': '',
   'area': 1361528.8281820714},
  {'SOIL_CODE': 'UCE',
   'CLASS': 'Other',
   'SUBCLASS': '',
   'area': 417291.5928367517},
  {'SOIL_CODE': 'URB',
   'CLASS': 'Other',
   'SUBCLASS': '',
   'area': 1473163.9603918842},
  {'SOIL

## Put the results on a map (notebook only)

In [15]:
m = folium.Map(location=[40.4820627,-79.9139233], tiles='Stamen Toner', zoom_start=13)

In [16]:
intersected_for_m = intersected.to_crs({'init': 'epsg:4326'})
intersected_for_m

,,,geometry
SOIL_CODE,CLASS,SUBCLASS,
GQF,Other,,(POLYGON ((-79.91230136440926 40.4755632821824...
GSF,Other,,(POLYGON ((-79.91450859818308 40.4843265797166...
GlC,3,Medium,POLYGON ((-79.90509537857614 40.48047413358914...
GpB,3,High,POLYGON ((-79.90618163113365 40.48266445286478...
GpD,Other,,(POLYGON ((-79.91677748187334 40.4843778529425...
UB,Other,,(POLYGON ((-79.90874534885515 40.4704974856890...
UCB,Other,,(POLYGON ((-79.90758440455558 40.4723354251644...
UCD,Other,,(POLYGON ((-79.91504003452576 40.4678736368951...
UCE,Other,,(POLYGON ((-79.90759749233224 40.4721817475412...


In [17]:
# folium.GeoJson(
#     parcels_gdf,
#     name='parcels'
# ).add_to(m)

# folium.GeoJson(
#     soils_gdf,
#     name='soils'
# ).add_to(m)

folium.GeoJson(
    intersected,
    name='intersected'
).add_to(m)

folium.LayerControl().add_to(m)

m